# Visualisering av ekstremverdier med dash apps

Notebooken inneholder to interaktive dash apps for visualisering av ekstremverdier 
og visualisering av statistiske verdier med og uten ekstremverdier.

## Felles import for alle kodeblokker

In [1]:
import calendar
import pandas as pd
import plotly.graph_objects as go
import sys

from dash import Dash, html, dcc, Input, Output

sys.path.append("../../src/analyseData")

from basedata import DataLoader
from yearlystats import YearlyStats
from outlieranalysis import OutlierAnalysis


"""Initialiser datalastere og statistikkobjekter"""
DATA_PATH = "../../data/processed"
loader = DataLoader(DATA_PATH)
ys = YearlyStats(DATA_PATH)
oa = OutlierAnalysis(DATA_PATH)

cities = ["oslo", "tromso"]
element_ids = [
    "sum(precipitation_amount P1D)",
    "mean(air_temperature P1D)",
    "mean(wind_speed P1D)",
    "max(air_temperature P1D)",
    "min(air_temperature P1D)",
    "range(air_temperature P1D)"
]

### Interaktiv versjon for visning av antall ekstrem verdier

In [2]:
app = Dash(__name__)
app.title = "Outlier-visualisering"

app.layout = html.Div(
    style={"maxWidth": "900px", "margin": "auto"},
    children=[
        html.H1("Ekstremverdier (Outliers) per måned"),
        html.Label("By:"),
        dcc.Dropdown(
            id="city-dropdown",
            options=[{"label": c.capitalize(), "value": c} for c in cities],
            value=cities[0],
            clearable=False,
        ),
        html.Label("Måletype (elementId):"),
        dcc.Dropdown(
            id="element-dropdown",
            options=[{"label": e, "value": e} for e in element_ids],
            value="sum(precipitation_amount P1D)",
            clearable=False,
        ),
        html.Label("Vis måneder uten outliers:"),
        dcc.Checklist(
            id="toggle-empty-months",
            options=[{"label": "Ja", "value": "vis"}],
            value=["vis"],
            inline=True,
        ),
        dcc.Graph(id="outlier-graph"),
    ],
)

@app.callback(
    Output("outlier-graph", "figure"),
    Input("city-dropdown", "value"),
    Input("element-dropdown", "value"),
    Input("toggle-empty-months", "value"),
)
def update_outlier_graph(city: str, element_id: str, show_empty_months: list[str]) -> go.Figure:
    """
    Oppdater grafen når brukeren endrer by, måletype eller visning.
    """
    include_empty = "vis" in show_empty_months
    offset = loader._get_min_offset(city, element_id)

    df_outliers = oa.find_outliers_per_month(
        city=city,
        element_id=element_id,
        time_offset=offset,
        include_empty_months=include_empty,
    )

    df_outliers["year_month"] = pd.to_datetime(df_outliers["year_month"])
    df_outliers["måned"] = df_outliers["year_month"].dt.strftime("%b %Y")

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df_outliers["måned"],
            y=df_outliers["outliers_removed"],
            name="Antall outliers",
            mode="lines+markers",
            marker=dict(size=6),
            line=dict(color="firebrick"),
        )
    )

    fig.update_layout(
        title=(
            f"Antall outliers per måned – {element_id} "
            f"i {city.capitalize()} (Offset {offset})"
        ),
        xaxis=dict(title="Måned", tickangle=45),
        yaxis=dict(title="Antall outliers"),
        template="plotly_white",
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8054)


### Interaktiv analyse av statistiske mål

In [3]:
app = Dash(__name__)
app.title = "Klimatologi med/uten outliers"

app.layout = html.Div(
    style={"maxWidth": "900px", "margin": "auto"},
    children=[
        html.H1("Klimatologisk Månedsmiddel"),

        html.Label("Måletype (elementId):"),
        dcc.Dropdown(
            id="element-dropdown",
            options=[{"label": e, "value": e} for e in element_ids],
            value=element_ids[0],
            clearable=False,
        ),

        html.Label("Statistikk:"),
        dcc.RadioItems(
            id="stat-radio",
            options=[
                {"label": "Gjennomsnitt", "value": "mean"},
                {"label": "Median", "value": "median"},
                {"label": "Std.avvik", "value": "std"},
            ],
            value="mean",
            inline=True,
        ),

        html.Label("Outliers:"),
        dcc.RadioItems(
            id="outlier-radio",
            options=[
                {"label": "Ta med", "value": "med"},
                {"label": "Uten", "value": "uten"},
            ],
            value="med",
            inline=True,
        ),

        html.Label("By(er):"),
        dcc.Checklist(
            id="city-checklist",
            options=[{"label": c.capitalize(), "value": c} for c in cities],
            value=cities,
            inline=True,
        ),

        dcc.Graph(id="climate-graph"),
    ],
)

@app.callback(
    Output("climate-graph", "figure"),
    Input("element-dropdown", "value"),
    Input("stat-radio", "value"),
    Input("outlier-radio", "value"),
    Input("city-checklist", "value"),
)
def update_graph(element_id, statistic, outlier_choice, selected_cities):

    fig = go.Figure()
    month_order = [calendar.month_abbr[m].capitalize() for m in range(1, 13)]
    remove = (outlier_choice == "uten")

    for city in selected_cities:
        df_climate = ys.climatological_monthly_mean(
            city,
            element_id,
            remove_outliers=remove,
            statistic=statistic,
        )
        df_climate = df_climate.sort_values("month")

        fig.add_trace(
            go.Bar(
                x=df_climate["month_name"],
                y=df_climate["value"],
                name=city.capitalize(),
            )
        )

    t_out = "uten outliers" if remove else "med outliers"
    fig.update_layout(
        title=f"Klimatologisk {statistic} per måned – {element_id} ({t_out})",
        xaxis_title="Måned",
        yaxis_title="Verdi",
        barmode="group",
        template="plotly_white",
        xaxis=dict(categoryorder="array", categoryarray=month_order),
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8051)